## Elixir Matrex library for linear regression MNIST digits recognizing

Based on Andrew Ng's Courser course on ML, excercise number 3.

In [1]:
# Define cost function with regularization.
# Take notion of heavy usage of paired functions, which do two matrix operations at one call.
# For example, dot_tn() returns dot product of two matrices, the first of which is being
# transposed. So, you get transposing operation here almost for free.
# And avoid one data copying operation in Elixir.

# Cost function returns cost value (scalar) for the given solution theta
# and gradient values (column matrix).
defmodule LinearRegression do
  def lr_cost_fun(%Matrex{} = theta, {%Matrex{} = x, %Matrex{} = y, lambda} = _params)
      when is_number(lambda) do
    m = y[:rows]

    h = Matrex.dot_and_apply(x, theta, :sigmoid)
    l = Matrex.ones(theta[:rows], theta[:cols]) |> Matrex.set(1, 1, 0)

    regularization =
      Matrex.dot_tn(l, Matrex.square(theta))
      |> Matrex.scalar()
      |> Kernel.*(lambda / (2 * m))

    # Compute the cost and add regularization parameter
    j =
      y
      |> Matrex.dot_tn(Matrex.apply(h, :log), -1)
      |> Matrex.substract(
        Matrex.dot_tn(
          Matrex.substract(1, y),
          Matrex.apply(Matrex.substract(1, h), :log)
        )
      )
      |> Matrex.scalar()
      |> (fn
            NaN -> NaN
            x -> x / m + regularization
          end).()

    # Compute gradient
    grad =
      x
      |> Matrex.dot_tn(Matrex.substract(h, y))
      |> Matrex.add(Matrex.multiply(theta, l), 1.0, lambda)
      |> Matrex.divide(m)

    {j, grad}
  end
  
  # The same cost function, implemented with  operators from `Matrex.Operators` module.
  # Works 2 times slower, than standard implementation. But it's a way more readable.
  # It is here for demonstrating possibilites of the library.
  def lr_cost_fun_ops(%Matrex{} = theta, {%Matrex{} = x, %Matrex{} = y, lambda} = _params)
      when is_number(lambda) do
    # Turn off original operators. Use this with caution!
    import Kernel, except: [-: 1, +: 2, -: 2, *: 2, /: 2, <|>: 2]
    import Matrex.Operators
    
    # This line is needed only when used from iex, to remove ambiguity of t/1 function.
    import IEx.Helpers, except: [t: 1]

    m = y[:rows]

    h = sigmoid(x * theta)
    l = ones(size(theta)) |> set(1, 1, 0.0)

    j = (-t(y) * log(h) - t(1 - y) * log(1 - h) + lambda / 2 * t(l) * pow2(theta)) / m

    grad = (t(x) * (h - y) + (theta <|> l) * lambda) / m

    {scalar(j), grad}
  end
end

{:module, LinearRegression, <<70, 79, 82, 49, 0, 0, 15, 144, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 132, 0, 0, 0, 45, 23, 69, 108, 105, 120, 105, 114, 46, 76, 105, 110, 101, 97, 114, 82, 101, 103, 114, 101, 115, 115, 105, 111, ...>>, {:lr_cost_fun_ops, 2}}

In [2]:
# Check working directory to know, where to load data files from.
File.cwd()

{:ok, "/Users/catalyst/Projects/Elixir/IElixir"}

In [3]:
# Load training data (5 000 MNIST digits in 20x20 pixels format)
# A column of ones was added to the beginning of the matrix.
x = Matrex.load("../matrex/test/X.mtx")

#Matrex[5000×401]
┌                                                                                     ┐
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     1.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0     0.0 │
│     ⋮       

In [4]:
# Load labels for each digit. 10 labels zero.
y = Matrex.load("../matrex/test/y.mtx")

#Matrex[5000×1]
┌         ┐
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│     ⋮   │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
└         ┘

In [5]:
# Initialize theta values. 
theta = Matrex.zeros(x[:cols], 1)

#Matrex[401×1]
┌         ┐
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     ⋮   │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
└         ┘

In [6]:
# Regularization parameter and number of learning iterations
lambda = 0.01
iterations = 100

100

In [7]:
# Start learning!
# For speed, learning is done in async streams. 
# It's 'one-vs-all' scheme for each digit.
solutions =
      1..10  # Our ten digits, we wish to recognize
      |> Task.async_stream(
        fn digit ->
          # Prepare labels matrix with only current digit labeled with 1.0
          y3 = Matrex.apply(y, fn val -> if(val == digit, do: 1.0, else: 0.0) end)

          # Use fmincg() optimizer (ported to Elixir with Matrex functions) with previously defined cost function.
          {sX, fX, _i} =
            Matrex.Algorithms.fmincg(&LinearRegression.lr_cost_fun/2, theta, {x, y3, lambda}, iterations)
        
          # Return the digit itself and the best found solution, which is a column matrix 401x1
          {digit, List.last(fX), sX}
        end,
        max_concurrency: 4
      ) # Merge all 10 found solution column matrices into one 10x401 solutions matrix
      |> Enum.map(fn {:ok, {_d, _l, theta}} -> Matrex.to_list(theta) end)
      |> Matrex.new()

Iterations 100 | Cost: 0.006114361251831055
Iterations 100 | Cost: 0.029583837524414063
Iterations 100 | Cost: 0.05683800833129883
Iterations 100 | Cost: 0.051797861007690425
Iterations 100 | Cost: 0.04844467248535156
Iterations 100 | Cost: 0.07778291082763672
Iterations 100 | Cost: 0.013563606567382811
Iterations 100 | Cost: 0.03457618876647949
Iterations 100 | Cost: 0.07014992135620117
Iterations 100 | Cost: 0.008773036560058595


#Matrex[10×401]
┌                                                                                     ┐
│-3.49315     0.0     0.0  1.4e-4 -0.0013  … -0.27188-0.06875 0.00849     0.0     0.0 │
│-3.45432     0.0     0.0 -5.0e-5  6.5e-4  …  0.01899 0.02443 0.00666 -7.6e-4     0.0 │
│-5.50862     0.0     0.0 -2.0e-5 -0.0007  … -0.01347 0.00136 -6.0e-5     0.0     0.0 │
│-2.39906     0.0     0.0 -1.0e-5 -3.0e-5  … -0.01055-0.00791  6.7e-4  2.0e-5     0.0 │
│ 0.62618     0.0     0.0     0.0  2.9e-4  … -0.00636 0.02062-0.00191 -1.0e-5     0.0 │
│-4.15499     0.0     0.0     0.0  3.0e-5  … -0.01228 0.00262 -3.7e-4  2.0e-5     0.0 │
│-1.98235     0.0     0.0 -3.0e-5  0.0002  … -0.00179-0.00233 -5.1e-4  6.0e-5     0.0 │
│-8.87112     0.0     0.0 -1.0e-5  1.1e-4  … -0.03071 0.00171 -2.4e-4  2.0e-5     0.0 │
│-5.32055     0.0     0.0     0.0  1.0e-5  …  0.01266  2.1e-4-0.00344  2.8e-4     0.0 │
│-5.81007     0.0     0.0     0.0 -1.0e-5  …  0.00554  9.8e-4 -2.9e-4  1.0e-5     0.0 │
└               

In [8]:
# Hope it was fast enough!
# Now let's check the quality of the solution.

# Apply solution to our learning data
predictions =
  x
  |> Matrex.dot_nt(solutions)
  |> Matrex.apply(:sigmoid)

#Matrex[5000×10]
┌                                                                                 ┐
│     0.0  1.7e-4  1.0e-5     0.0  0.0001     0.0     0.0  1.0e-5 0.00155 0.99948 │
│     0.0  1.0e-5  1.0e-5     0.0 0.00812     0.0     0.0     0.0  2.0e-5 0.99992 │
│     0.0  0.0003  2.5e-4     0.0  7.0e-5     0.0     0.0 0.00772  7.1e-4 0.99949 │
│     0.0  2.4e-4  1.0e-5     0.0     0.0  1.8e-4     0.0  6.0e-5  1.1e-4 0.99993 │
│     0.0     0.0  1.0e-5     0.0  5.1e-4     0.0     0.0  3.0e-5     0.0 0.99914 │
│     0.0  1.5e-4     0.0     0.0     0.0     0.0     0.0 0.02063     0.0     1.0 │
│     0.0     0.0  0.0364     0.0     0.0     0.0     0.0 0.00159  2.0e-5 0.99912 │
│     0.0  0.3803  8.8e-4     0.0 0.17375     0.0     0.0 0.00334     0.0 0.95394 │
│     0.0  1.8e-4  0.0003     0.0  8.0e-5     0.0  1.0e-5 0.05938  1.3e-4 0.99934 │
│     0.0     0.0 0.24179     0.0  4.4e-4     0.0  1.0e-5  1.0e-5 0.00281 0.99916 │
│     ⋮       ⋮       ⋮       ⋮       ⋮       ⋮       ⋮    

In [9]:
# Each row of predictions matrix contains probability that corresponding row of training data X
# contains the image of digit, which is equal to this row's index.
# I.e. algorithm predicts, that first row contains zero,
# beacuse the tenth cell of the row contains the greatest value (0.99952), than other cells.
# And label 10 represents zero in our data.

# Now let's take these predictions, compare them to our labels and count all 
# true predictions (i.e. where prediction for the row is equal to it's label.)
# Then divide this number by the total number of predictions and multiply by 100
# to get the percentage of true predictions, and thus, the accuracy of our learning algorithm.

accuracy =
  1..predictions[:rows]
  |> Enum.reduce(0, fn row, acc ->
    if y[row] == predictions[row][:argmax], do: acc + 1, else: acc
  end)
  |> Kernel./(predictions[:rows])
  |> Kernel.*(100)

95.89999999999999

In [10]:
# Not bad!

nil